<a href="https://colab.research.google.com/github/odarasimi/Cryptocurrency-RNN/blob/master/Cryptocurrency_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import io
from sklearn import preprocessing
from collections import deque
import random
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Dense, Dropout, LSTM
from keras.layers import MaxPooling2D


In [ ]:
from google.colab import files
bch_usd = files.upload()

In [ ]:
from google.colab import files
btc_usd = files.upload()

In [ ]:
from google.colab import files
eth_usd = files.upload()

In [ ]:
from google.colab import files
ltc_usd = files.upload()

In [8]:
df_BCH = pd.read_csv(io.BytesIO(bch_usd["BCH-USD.csv"]), names=["time", "low", "high", "open", "close", "volume"]) 
df_BCH.set_index("time", inplace=True)
df_BTC = pd.read_csv(io.BytesIO(btc_usd["BTC-USD.csv"]), names=["time", "low", "high", "open", "close", "volume"])
df_BTC.set_index("time", inplace=True)
df_ETH = pd.read_csv(io.BytesIO(eth_usd["ETH-USD.csv"]), names=["time", "low", "high", "open", "close", "volume"]) 
df_ETH.set_index("time", inplace=True)
df_LTC= pd.read_csv(io.BytesIO(ltc_usd["LTC-USD.csv"]), names=["time", "low", "high", "open", "close", "volume"])
df_LTC.set_index("time", inplace=True)

In [9]:
df_BCH.head()

,low,high,open,close,volume
time,,,,,
1528968660,871.650024,871.729980,871.650024,871.719971,5.675361
1528968720,870.859985,871.719971,871.719971,870.859985,26.856577
1528968780,870.099976,871.090027,871.090027,870.099976,1.124300
1528968840,868.830017,870.950012,868.830017,870.789978,1.749862
1528968900,870.000000,870.000000,870.000000,870.000000,1.680500


In [10]:
new_df = pd.DataFrame()
cryptos = [[df_BCH,"df_BCH"], [df_BTC,"df_BTC"], [df_ETH,"df_ETH"], [df_LTC,"df_LTC"]]
for crypto in cryptos:
  crypto[0].rename(columns={"close": f"{crypto[1]}_close", "volume": f"{crypto[1]}_volume"}, inplace=True)
  df = crypto[0][[f"{crypto[1]}_close", f"{crypto[1]}_volume"]]

  if len(new_df) == 0:
    new_df = df
  else:
    new_df = new_df.join(df)
new_df.head()

,df_BCH_close,df_BCH_volume,df_BTC_close,df_BTC_volume,df_ETH_close,df_ETH_volume,df_LTC_close,df_LTC_volume
time,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978


In [11]:
new_df.fillna(method="ffill", inplace=True)  # if there are any gaps in data, we use previously known values
new_df.dropna(inplace=True)
seq_len = 60
future_predict = 3

crypto_predict = "df_LTC"

In [12]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [13]:
new_df['future'] = new_df[f'{crypto_predict}_close'].shift(-future_predict)
new_df['target'] = list(map(classify, new_df[f'{crypto_predict}_close'], new_df['future']))


In [14]:
new_df.head()

,df_BCH_close,df_BCH_volume,df_BTC_close,df_BTC_volume,df_ETH_close,df_ETH_volume,df_LTC_close,df_LTC_volume,future,target
time,,,,,,,,,,
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024,96.389999,0
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799,96.519997,0
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067,96.440002,0
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978,96.470001,1
1528968960,869.989990,1.669014,6480.000000,1.490900,486.00000,7.503300,96.519997,16.991997,96.400002,0


In [15]:
times = sorted(new_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_df = new_df[(new_df.index >= last_5pct)]
new_df = new_df[(new_df.index < last_5pct)]

In [16]:
def preprocess(df):
  df = df.drop("future", 1)
  for col in df.columns:
    if col != "target":
      df[col] = df[col].pct_change()
      df.dropna(inplace = True)
      df[col] = preprocessing.scale(df[col].values)

  df.dropna(inplace=True)
  sequential_data = []
  prev_days = deque(maxlen=seq_len)
  for i in df.values:
    prev_days.append([n for n in i[:-1]])
    if len(prev_days) == seq_len:
      sequential_data.append([np.array(prev_days), i[-1]])

  buys = []
  sells = []

  for seq, target in sequential_data:
    if target == 0:
      sells.append([seq,target])
    else:
      buys.append([seq,target])

  random.shuffle(buys)
  random.shuffle(sells)

  lower = min(len(buys), len(sells))
  buys = buys[:lower]
  sells = sells[:lower]

  sequential_data = buys + sells
  random.shuffle(sequential_data)

  X = []
  y = []
  for seq, target in sequential_data:
    X.append(seq)
    y.append(target)

  return np.array(X), y



  

**Test the preprocess function**

In [17]:
X_train, y_train = preprocess(new_df)
X_test, y_test = preprocess(validation_df)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

print(f"train data: {len(X_train)}")
  

train data: 75008


**Begin training of neural network**

In [19]:
model = Sequential()

model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(128))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [20]:
#alt_opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.fit(X_train, y_train, batch_size = 50, epochs = 10)

Epoch 1/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6906 - accuracy: 0.5292
Epoch 2/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6830 - accuracy: 0.5598
Epoch 3/10
1501/1501 [==============================] - 20s 13ms/step - loss: 0.6809 - accuracy: 0.5648
Epoch 4/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6798 - accuracy: 0.5673
Epoch 5/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6782 - accuracy: 0.5720
Epoch 6/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6757 - accuracy: 0.5763
Epoch 7/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6715 - accuracy: 0.5840
Epoch 8/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6662 - accuracy: 0.5944
Epoch 9/10
1501/1501 [==============================] - 19s 13ms/step - loss: 0.6565 - accuracy: 0.6093
Epoch 10/10
1501/1501 [==============================] - 19s 13m